In [ ]:
import numpy as np
import gymnasium as gym

# Your custom environment setup
desc = ["SFFF", "FHHH", "FFFF", "HFHF", "FFGF"]
env = gym.make('FrozenLake-v1', desc=desc, map_name="5x5", is_slippery=False, render_mode="human")

# Initialize Q-table with zeros
Q = np.zeros([env.observation_space.n, env.action_space.n])

# Hyperparameters
# Adjust hyperparameters
# Learning parameters
alpha = 0.1  # Consider experimenting with this, possibly increasing up to 0.2 or 0.3
gamma = 0.99  # High gamma to value future rewards, consider experimenting within 0.9 to 0.99
epsilon = 1.0  # Initial exploration rate
epsilon_min = 0.01  # Lower minimum to ensure some level of exploration continues
epsilon_decay = 0.995  # Adjust this to control the rate of decay, ensuring enough exploration
learning_episodes = 20000  # Increase if the environment and task are complex

# Penalty values, adjusted to be less severe
step_penalty = -0.001
no_progress_penalty = -0.01

# Initialize Q-table
Q = np.zeros([env.observation_space.n, env.action_space.n])

for episode in range(1, learning_episodes + 1):
    state, _ = env.reset()
    done = False
    total_reward = 0  # Keep track of the total reward

    while not done:
        if np.random.uniform(0, 1) < epsilon:
            action = env.action_space.sample()  # Explore action space
        else:
            action = np.argmax(Q[state])  # Exploit learned values

        next_state, reward, done, _, info = env.step(action)
        
        # Apply step penalty to encourage finding the goal faster
        reward += step_penalty

        # Check if the agent didn't make progress (e.g., moved into a wall)
        if next_state == state:
            reward += no_progress_penalty

        # Q-learning update
        old_value = Q[state, action]
        next_max = np.max(Q[next_state])
        new_value = (1 - alpha) * old_value + alpha * (reward + gamma * next_max)
        Q[state, action] = new_value

        state = next_state
        total_reward += reward

    # Decay epsilon
    epsilon = max(epsilon_min, epsilon_decay * epsilon)

    if episode % 100 == 0:
        print(f"Episode: {episode}, Total Reward: {total_reward}")

print("Training finished.")